### docTR + OpenAI API

The moitve behind this project is when I saw my wife spent hours preparing exercice to our son. After diner, she seat at the couch with me and she started reading his textbook and prepare true/false and fill the blanks questions.

So, why not automate this job? I tested ChatGPT giving a text and ask it to create 10 true/false and fill the blanks qustions. It showed me the result, and it was exactly what I wanted. The next step is to find a way to translate the textbook into a digitial format. Unfortunelty, we did not have the pdf of his textbook. So, the solutino is to take a picture and somehow translate into text file.

That's where docTR comes in. docTR is an open soure Optical Character Recognition (OCR) that recognize words in an image and translate into a text format.

Below is a simple proof of concept of this solution.

Please not that this is an on-going project. I will improve the process based on our needs.

### Import all libraries

In [195]:
import pandas as pd
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import json
import os
import glob
import re

from dotenv import dotenv_values
import openai

### Constants

In [222]:
IMAGE_PATH = ".//images//"
OUTPUT_TEXT_FROM_IMAGE = ".//text_from_images//"
EXAM_QUESTIONS_DIR = ".//exam_questions//"

### Functions

In [145]:
def get_images(path=IMAGE_PATH, patter="*"):
    """
    Return a list with all files in the directory
    Parameters:
        path (string): the directory where the files are stored
    """
    #files = sorted(os.walk(path))
    files = glob.glob(path + patter)
    return files

In [165]:
def write_file(file_name, text_to_file, path=OUTPUT_TEXT_FROM_IMAGE, extension=".txt"):
    file = open(path + file_name + extension, "w")
    file.write(text_to_file)
    file.close()

In [125]:
def extract_text(text_json_format):
    final_text = ""
    for obj1 in text_json_format["pages"][0]["blocks"]:
        final_text += " \n "
        for obj2 in obj1["lines"]:
            #final_text += " \n "
            for obj3 in obj2["words"]:
                final_text += " " + obj3['value']
    return final_text

In [229]:
def generate_exame(text, exam_type="True or False", number_questions=None):
    """
    
    Parameters:
        text (str): The text that ChatGPT will use to generate the exam.
        exam_type (str): type of exam. It could be 'fill the blanks', 'true or False', 'multipe questions'
    """
    #if (number_questions is not None):
    #    number_question_text = f"{number_questions}"
    system_msg = f"You are a teacher specialized in history of Canada. \
                  I will send you a text regarding Canadian history, \
                  and I would like that you create an exam for a 10 years old kid \
                  The gola is to practice to his/her next exame at school. \
                  So, please generate from follwoing text, {number_questions} {exam_type} style exam: {text} \
                  Also provide the anwsers for each question at the and. \
                  "
    messages = [
        {"role": "system", "content": system_msg 
        }
    ]  
    
    response = openai.ChatCompletion.create(
        messages = messages,
        model = "gpt-3.5-turbo",
        max_tokens = 500,
    )
    exam = response["choices"][0]["message"]["content"]
    return exam

### Main flow

In [184]:
config = dotenv_values("../env.txt")
openai.api_key = config["OPENAI_KEY"]

In [219]:
files = get_images()
files

['.//images/pag1_2.jpg', './/images/pag1_1.jpg']

In [126]:
# Model instance
# model = ocr_predictor(pretrained=True, detect_language=)
model = ocr_predictor('db_resnet50', 'crnn_vgg16_bn', pretrained=True, assume_straight_pages=True)

/Users/carlacosta/Alberto/Pandas-Machine-Learning-Central/Projects/Question-Creator-School/env/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/carlacosta/Alberto/Pandas-Machine-Learning-Central/Projects/Question-Creator-School/env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [176]:
# loop for all images and generate the text file
text = ""
# get the base file name for the final file with all text combined
text_file_name = os.path.basename(file_name).split(".")[0]
for file_name in files:
    print(f"Converting the image file {os.path.basename(file_name)}...")
    document = DocumentFile.from_images(file_name)
    result = model(document)
    json_export = result.export()
    text += extract_text(json_export)

write_file(file_name=text_file_name, text_to_file=setattr(text)
    
print(f"File '{text_file_name}' created under {OUTPUT_TEXT_FROM_IMAGE}")

Converting the image file pag1_2.jpg...
Converting the image file pag1_1.jpg...
File pag1_1 created under .//text_from_images//


In [226]:
# generate the exam
exam = generate_exame(text=text, exam_type="True or False", number_questions=3)

In [227]:
write_file(file_name="exam_"+text_file_name, path=EXAM_QUESTIONS_DIR, text_to_file=exam)